![](https://thumbs.gfycat.com/FeistyNaughtyFinch-size_restricted.gif)
# Predicting rain in Australia

This week, we are working with the dataset to predict the possibility of raining the next day. Studies in 2006 showed that weather information had an average value of 109 dollars per US household each year. This equates to an overall economic benefit of over 12 billion dollars per annum. Being able to forecast and plan for the future when it comes to the local climate is a major advantage when it comes to planning tourism facilities. The transport sector can also benefit, as infrastructure can be set up to measure road surface conditions to improve traffic safety.

In [ ]:
# Importing required packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
# Creating a function to print 
def overview():
    data =pd.read_csv('../input/weather-dataset-rattle-package/weatherAUS.csv')
    print("First 5 lines of data:\n")
    print(data.head())
    print("\n\n\n")
    print("There are {} rows and {} columns".format(data.shape[0], data.shape[1]))
    print("\n\n\n")
    print("Data types:\n")
    print(data.dtypes)
    print("\n\n\n")
    print("% of missing values per column:\n")
    print(data.isnull().mean().round(2)*100)
    print("Statistical summary:\n")
    print(data.describe())
    return data
    
data = overview()

- It seems like we have quite alot of data cleaning to do (dealing with NaN values).
- For columns with less than 5% missing values, we will remove the affected rows.
- For columns with more than 5% missing values, we will fill it with median (numerical data) and mode (categorical data).
- As stated in the dataset description, we will drop the column call "RISK_MM" since it will affect our forcasting.

In [ ]:
data.drop(columns = "RISK_MM", inplace = True)

## Looking into distribution for RainTomorrow

In [ ]:
sns.countplot(data = data, x = "RainTomorrow")

## Looking into distribution for RainToday

In [ ]:
sns.countplot(data = data, x = "RainToday")

## Looking into min temp and max temp

In [ ]:
sns.distplot(data['MinTemp'])

In [ ]:
sns.distplot(data['MaxTemp'])

- We can see that the min temperature is clustered around 10 while max temperature is around 20. 

## Pairplot

In [ ]:
# Create a function to separate out numerical and categorical data 
    ## Using this function to ensure that all non-numerical in a numerical column 
    ## and non-categorical in a categorical column is annotated
def cat_variable(df):
    return list(df.select_dtypes(include = ['category', 'object']))
    
def num_variable(df):
    return list(df.select_dtypes(exclude = ['category', 'object']))

categorical_variable = cat_variable(data)
numerical_variable = num_variable(data)

In [ ]:
sns.pairplot(data[numerical_variable], kind='scatter', diag_kind='hist', palette='Rainbow')
plt.show()

In [ ]:
# Dealing with outliers

q = data[numerical_variable].quantile(0.99)
data_new = data[data[numerical_variable]< q]

### Re-adjusting categorical variables to numeric variables

In [ ]:
data = data.dropna(subset=['Rainfall', 'WindDir3pm', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm', 'Temp9am','Temp3pm','RainToday'])
cat_variable = ['WindGustDir', 'WindDir9am']
num_variable = ['MinTemp', 'MaxTemp', 'Evaporation', 'Sunshine', 'WindGustSpeed', 'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm']
imp_cat = SimpleImputer(missing_values = np.nan, strategy='most_frequent')
data[cat_variable] = imp_cat.fit_transform(data[cat_variable])
imp_num = SimpleImputer(missing_values = np.nan, strategy='median')
data[num_variable] = imp_num.fit_transform(data[num_variable])

In [ ]:
le = LabelEncoder()
 
# Implementing LE on WindGustDir
le.fit(data.WindGustDir.drop_duplicates()) 
data.WindGustDir = le.transform(data.WindGustDir)

# Implementing LE on WindDir9am
le.fit(data.WindDir9am.drop_duplicates()) 
data.WindDir9am = le.transform(data.WindDir9am)

# Implementing LE on WindDir3pm
le.fit(data.WindDir3pm.drop_duplicates()) 
data.WindDir3pm = le.transform(data.WindDir3pm)

# Implementing LE on RainToday
le.fit(data.RainToday.drop_duplicates()) 
data.RainToday = le.transform(data.RainToday)

# Implementing LE on RainTomorrow
le.fit(data.RainTomorrow.drop_duplicates()) 
data.RainTomorrow = le.transform(data.RainTomorrow)

In [ ]:
plt.figure(figsize=(15,10))
 
corrMatrix = data.corr()
sns.heatmap(corrMatrix, annot=True)
plt.show()

## Implementing logistic regression 

In [ ]:
# Assigning X and y
X = data.drop(['RainTomorrow', 'Date', 'Location'], axis=1)

y = data['RainTomorrow']

# Implementing train and test splits
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
# Looking into the shape of training and test dataset
print(X_train.shape)
print(X_test.shape)

In [ ]:
# instantiate the model
logreg = LogisticRegression(solver='liblinear', random_state=0)


# fit the model
logreg.fit(X_train, y_train)

In [ ]:
y_pred_test = logreg.predict(X_test)
print('Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred_test)))